In [ ]:
import numpy as np
import pandas as pd

import matplotlib        as mpl
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation

from sklearn.datasets        import load_boston
from sklearn.metrics         import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import MinMaxScaler

from IPython.display         import HTML


The steps involved in solving a problem with machine learning techniques-
1. data preprocessing
2. define 'error'
3. split the data into two parts- trainig and testing data
4. train the model
5. prediction: obtain the predicted values for the given input and vizualize the results.


STEP 1 - DATA PREPROCESSING

In [ ]:
#load the dataset
boston=load_boston()

#description of datdset
print(boston.DESCR)

In [ ]:
#put the data into pandas dataframes
features = pd.DataFrame(boston.data,columns=boston.feature_names)
features

In [ ]:
features['AGE']

In [ ]:
target = pd.DataFrame(boston.target,columns=['target'])
target

In [ ]:
max(target['target'])

In [ ]:
min(target['target'])

In [ ]:
# concatenate features and target into a simple dataframe
#axis = 1 makes it concatenate column wise
df = pd.concat([features,target],axis=1)
df

In [ ]:
df.describe().round(decimals =2)

In [ ]:
#calculate correlation between every on the data
corr = df.corr('pearson')

#take absolute values of correlation
corrs =[abs(corr[attr]['target']) for attr in list (features)]

#make a list of pairs [(corr, feature)]
l = list(zip(corrs, list(features)))

#sort the list of pairs in reverse/descending order
# with the correlation value as the key for sorting
l.sort(key = lambda x : x[0], reverse=True)

# unzip pairs to two lists
# zip(*l) - takes a list that looks like [[a,b,c],[d,e,f],[g,h,i]]
# and returns [[a,d,g],...]
corrs, labels = list(zip((*l)))

#plot correlations with respect to the target variable as a bar graph
index = np.arange(len(labels))
plt.figure(figsize=(15,5))
plt.bar(index, corrs, width = 0.5)
plt.xlabel('Attributes')
plt.ylabel('Correlation with the target variable')
plt.xticks(index, labels)
plt.show()

NORMALIZE THE DATA

In [ ]:
X = df['DIS'].values
Y = df['target'].values

In [ ]:
#before noramlization
print(Y[:5])

In [ ]:
x_scaler= MinMaxScaler()
X = x_scaler.fit_transform(X.reshape(-1,1))
X = X[:,-1]
y_scaler = MinMaxScaler()
Y = y_scaler.fit_transform(Y.reshape(-1,1))
Y = Y[:, -1]

In [ ]:
# after normalization
print(Y[:5])

STEP 2 - DEFINE ERROR

In [ ]:
def error(m, x, c, t):
    N = x.size
    e = sum(((m * x + c ) - t) **2)
    return e * 1/(2 * N)

STEP 3 - SPLIT THE DATA ( training and testing )

In [ ]:
# 0.2 indicates 20% of the data is randomly sampled as testing data
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2)

UPDATE FUNCTION

In [ ]:
def update(m, x, c, t, learning_rate):
    grad_m = sum(2 * ((m * x + c) - t) * x)
    grad_c=sum(2 * ((m * x + c) - t))
    m = m - grad_m * learning_rate
    c = c - grad_c * learning_rate
    return m,c

Error threshold - if the error falls below the threshold, the gradient descent process is stopped and the weights are returned.

In [ ]:
def gradient_descent(init_m, init_c, x, t, learning_rate, iterations, error_threshold):
    m = init_m
    c = init_c
    error_values = list()
    mc_values = list()
    for i in range(iterations):
        e = error(m, x, c, t)
        if e < error_threshold:
            print('error less than the threshold. stopping gradient descent')
            break
        error_values.append(e)
        m, c = update(m, x, c, t, learning_rate)
        mc_values.append((m, c))
    return m, c, error_values, mc_values

learning rate- increasing the lr reduces the convergence time. But, if the learning rate is too high the model will overshoot the minima.

iterations- no. of iterations must be large enough to allow the model to converge to a minima, but if it is too large, then the model becomes too specific to the training data thus causing overfitting, i.e, the model 'memorizes' the data instead of 'learning' the data.
for this dataset if lr > 0.0025, it causes overflow. reduce the learning rate and observe the error values.

error threshold- this value can be set to a max. value of erroe that is acceptable. when the error values goes below the threshold, the gradient descent is stopped.


In [ ]:
%%time
init_m = 0.9
init_c = 0
learning_rate = 0.001
iterations = 250
error_threshold = 0.001

m, c, error_values, mc_values = gradient_descent(init_m, init_c, xtrain, ytrain, learning_rate, iterations, error_threshold)

In [ ]:
# as the number of iterations inreases, changes in the line are less noticable.
# inorder to reduce the processing time fot the animation, it is advised to choose smaller values
mc_values_anim = mc_values[0:250:5]


In [ ]:
fig, ax = plt.subplots()
ln, = plt.plot([], [], 'ro-', animated=True)

def init():
    plt.scatter(xtest, ytest, color='g')
    ax.set_xlim(0, 1.0)
    ax.set_ylim(0, 1.0)
    return ln,

def update_frame(frame):
    m , c = mc_values_anim[frame]
    x1, y1 = -0.5, m * -.5 + c
    x2, y2 = 1.5, m * 1.5 + c
    ln.set_data([x1, x2], [y1, y2])
    return ln,

anim = FuncAnimation(fig, update_frame, frames=range(len(mc_values_anim)),
                     init_func=init, blit=True)

HTML(anim.to_html5_video())

plotting the regressionline upon the training data set

In [ ]:
plt.scatter(xtrain, ytrain, color='b')
plt.plot(xtrain, (m * xtrain + c), color='r')

plotting error values

In [ ]:
plt.plot(np.arange(len(error_values)), error_values)
plt.ylabel('Error')
plt.xlabel('Iterations')


PREDICTION

In [ ]:
#calculate the predictions on the test set as a vectorized operation
predicted = (m * xtest) + c

In [ ]:
#compute MSE for the predicted values on the testing set
mean_squared_error(ytest, predicted)

In [ ]:
#put xtest, ytest and predicted values into a single DataFrame so that we
# can see the predicted values alongside the testing set
p = pd.DataFrame(list(zip(xtest, ytest, predicted)), columns=['x','target_y', 'predicted_y'])
p.head()


PLOT THE PREDICTED VALUES AGAINST THE TARGET VALUES


In [ ]:
plt.scatter(xtest, ytest, color='b')
plt.plot(xtest, predicted, color='r')


.
.
REVERT NORMALIZATION
.
.

In [ ]:
# reshape to change the shape to the shape that is required by the scalar
predicted = predicted.reshape(-1, 1)
xtest = xtest.reshape(-1, 1)
ytest = ytest.reshape(-1 ,1)

xtest_scaled = x_scaler.inverse_transform(xtest)
ytest_scaled = y_scaler.inverse_transform(ytest)
predicted_scaled = y_scaler.inverse_transform(predicted)

# this is to remove the extra dimnesion
xtest_scaled = xtest_scaled[:, -1]
ytest_scaled = ytest_scaled[:, -1]
predicted_scaled = predicted_scaled[:, -1]

p = pd.DataFrame(list(zip(xtest_scaled, ytest_scaled,predicted_scaled)), columns=['x','target_y','predicted_y'])
p = p.round(decimals = 2)
p
